## YouTube Trending Video Analysis & Prediction

### Joel Day, Caleb McCurdy, & Lane Whitmore

### The University of San Diego

#### ADS 508 - Data Science Cloud Computing

In [2]:
# General imports
import pandas as pd
import numpy as np

# nlp processing imports
import string
import re
import nltk

# Visualization imports
import matplotlib.pyplot as plt
import seaborn as sns

# Sagemaker imports
import sagemaker
# from sagemaker.sklearn.processing import 
from sagemaker import get_execution_role
from sklearn.feature_extraction.text import CountVectorizer

# athena connection imports
!pip install pyathena
from pyathena import connect
import boto3

## Establishing Athena for SQL Queries

In [13]:
region = boto3.Session().region_name
role = get_execution_role()

In [14]:
# Set Athena for data wrangling // exploration
session = sagemaker.Session()

bucket = "ads508youtubebucket"

In [15]:
ingest_create_athena_db_passed = False

db_name = "youtubedb"

In [16]:
# temp query storage
s3_athena = "s3://ads508youtubebucket/athena/tmp/"

# file storage
youtube_dir = "s3://ads508youtubebucket/athena/files/"

# creating connection to database
conn = connect(region_name=region, s3_staging_dir=s3_athena)

### Creating Database

In [17]:
# Creating DATABASE with db_name above
create_db = "CREATE DATABASE IF NOT EXISTS {}".format(db_name)

pd.read_sql(create_db, conn)

""


### Creating Tables

#### CSV Tables

Currently, the tables below are still struggling to appropriately populate the tables. If the data type is anything but strings, the column becomes null for all values and the comma seperation is not consistent.

In [56]:
pd.read_sql(f"DROP TABLE IF EXISTS {db_name}.ca_csv", conn)


ca_csv_create = f"""
CREATE EXTERNAL TABLE IF NOT EXISTS {db_name}.ca_csv(
                video_id string,
                title string,
                publishedAt string,
                channelId string,
                channelTitle string,
                categoryId string,
                trending string,
                tags string,
                view_count string,
                likes string,
                dislikes string,
                comment_count string,
                thumbnail_link string,
                comments_disabled string,
                ratings_disabled string,
                description string)
            
                
                ROW FORMAT SERDE 'org.apache.hadoop.hive.serde2.OpenCSVSerde' 
                WITH SERDEPROPERTIES ("separatorChar" = ",") 
                LOCATION 's3://ads508youtubebucket/youtubeStatistics/csv/can/'
                TBLPROPERTIES ("skip.header.line.count"="1");
"""

pd.read_sql(ca_csv_create, conn)

""


In [57]:
pd.read_sql(f"SELECT * FROM {db_name}.ca_csv LIMIT 10", conn)

,video_id,title,publishedat,channelid,channeltitle,categoryid,trending,tags,view_count,likes,dislikes,comment_count,thumbnail_link,comments_disabled,ratings_disabled,description
0,d1HUFPBWUh4,"Nous servons dans la cantine d'un lycée, les é...",2022-06-19T08:03:37Z,UCtxhwJ-6n4_C6ElALThD25Q,Mcfly et Carlito la chaîne secondaire,22,2022-06-21T00:00:00Z,[None],1684938,122259,0,1934,https://i.ytimg.com/vi/d1HUFPBWUh4/default.jpg,False,False,Après avoir perdu face à Maxime Gasteuil dans ...
1,DPJL488cfRw,Yunchan Lim 임윤찬 – RACHMANINOV Piano Concerto N...,2022-06-19T19:04:24Z,UC5Oz2fuXlILwEH4P96h2LLg,The Cliburn,10,2022-06-21T00:00:00Z,[None],1158892,36617,0,2125,https://i.ytimg.com/vi/DPJL488cfRw/default.jpg,False,False,"Final Round Concerto IIJune 17, 2022Bass Perfo..."
2,ASMMDYLry80,48hrs on Luxury Rocky Mountaineer Train - From...,2022-06-19T21:44:53Z,UC50xPFPma-nCS5GZpAFukpQ,Trek Trendy,19,2022-06-21T00:00:00Z,rocky mountaineer|rocky mountaineer train|luxu...,183186,7228,0,608,https://i.ytimg.com/vi/ASMMDYLry80/default.jpg,False,False,Head to https://squarespace.com/trektrendy to ...
3,Mbo4FzRy7k4,Everything in moderation! WHAT I EAT | Wellnes...,2022-06-20T04:48:12Z,UCAHufvd02viJSRdt3ojdPOg,Sarahs Day,22,2022-06-21T00:00:00Z,what I eat in a day|full day of eating|grocery...,178079,8414,0,331,https://i.ytimg.com/vi/Mbo4FzRy7k4/default.jpg,False,False,WELLNESS WEEK IS NOW LIVE!!https://www.sarahsd...
4,nEFcbKF4WfY,Return to Valoran City - Star Guardian 2022 | ...,2022-06-20T15:00:25Z,UC2t5bjwHdUX4vM2g8TRDq5g,League of Legends,20,2022-06-21T00:00:00Z,Star guardian|star guardian 2022|star guardian...,1000267,55422,0,3145,https://i.ytimg.com/vi/nEFcbKF4WfY/default.jpg,False,False,A new day is just over the horizon. Star Guard...
5,Ul_LSTpBVVA,Overpowered Weapons vs Custom Bosses in Minecraft,2022-06-19T15:30:03Z,UC_cvTMeip9po2hZdF3aBXrA,BeckBroJack,20,2022-06-21T00:00:00Z,minecraft|custom bosses|minecraft bosses|custo...,457641,38378,0,604,https://i.ytimg.com/vi/Ul_LSTpBVVA/default.jpg,False,False,Today I'm using Overpowered Weapons to beat Tw...
6,8kzU3gF92o0,ON DEVINE QUI EST SON MEC (Ft. Toomuchlucile &...,2022-06-19T15:30:01Z,UCXqnG7hbmGYDqC1iTNkVdew,Océane,22,2022-06-21T00:00:00Z,oceane|océane|océane les anges|oce.rthn|oce,729434,61449,0,2933,https://i.ytimg.com/vi/8kzU3gF92o0/default.jpg,False,False,"COUCOU GUYS!! Dans la vidéo d'aujourd'hui, on ..."
7,adBqwKLyZj4,I Completed The Wild Update Already..,2022-06-19T17:00:01Z,UCS5Oz6CHmeoF7vSad0qqXfw,DanTDM,20,2022-06-21T00:00:00Z,dantdm|dan tdm|gaming|video games|minecraft|funny,827033,45921,0,3312,https://i.ytimg.com/vi/adBqwKLyZj4/default.jpg,False,False,SUBSCRIBE OR THE WARDEN HUGS YOU TIGHTLYIn thi...
8,jQI5jD6ujXs,JEU EN VRAC - l'histoire de l'AMSTRAD,2022-06-19T16:00:32Z,UCCMxHHciWRBBouzk-PGzmtQ,Bazar du Grenier,20,2022-06-21T00:00:00Z,Jeu en vrac|jdg|joueur du grenier|bazar du gre...,604228,53875,0,1595,https://i.ytimg.com/vi/jQI5jD6ujXs/default.jpg,False,False,Cette vidéo est sponsorisée par Rhinoshield ! ...
9,D3qFfAEp9AU,Are They All The Same Motor Oil? Let's Settle ...,2022-06-19T16:49:32Z,UC2rzsm1Qi6N1X-wuOg_p0Ng,Project Farm,26,2022-06-21T00:00:00Z,project farm|motor oil review|are all motor oi...,533327,29212,0,4457,https://i.ytimg.com/vi/D3qFfAEp9AU/default.jpg,False,False,Let's find out if Pennzoil's motor oil (Pennzo...


In [48]:
pd.read_sql(f"DROP TABLE IF EXISTS {db_name}.us_csv", conn)


us_csv_create = f"""
CREATE EXTERNAL TABLE IF NOT EXISTS {db_name}.us_csv(
                video_id string,
                title string,
                publishedAt string,
                channelId string,
                channelTitle string,
                categoryId string,
                trending string,
                tags string,
                view_count string,
                likes string,
                dislikes string,
                comment_count string,
                thumbnail_link string,
                comments_disabled string,
                ratings_disabled string,
                description string)
            
                
                ROW FORMAT SERDE 'org.apache.hadoop.hive.serde2.OpenCSVSerde' 
                WITH SERDEPROPERTIES ("separatorChar" = ",") 
                LOCATION 's3://ads508youtubebucket/youtubeStatistics/csv/usa/'
                TBLPROPERTIES ("skip.header.line.count"="1");
"""

pd.read_sql(us_csv_create, conn)

""


In [49]:
pd.read_sql(f"SELECT * FROM {db_name}.us_csv LIMIT 10", conn)

,video_id,title,publishedat,channelid,channeltitle,categoryid,trending,tags,view_count,likes,dislikes,comment_count,thumbnail_link,comments_disabled,ratings_disabled,description
0,Y4wWqz3ng_U,This Kinda SUCKS! - Cyberpunk 2077 Next Gen Up...,2022-02-15T18:03:04Z,UCiHFS5txB0O7jckRk-oK8XQ,DreamcastGuy,20,2022-02-19T00:00:00Z,xbox|pc|one|sony|playstation|dreamcast|gaming|...,227405,4959,0,2621,https://i.ytimg.com/vi/Y4wWqz3ng_U/default.jpg,False,False,Like and subscribe if you enjoyed it!Follow me...
1,zk2rxLH80XA,The SHOCKING Truth about Yae Miko! 4★ Weapon C...,2022-02-16T02:52:38Z,UC8BTtymnCZ7jTvt09cjPMwQ,Gacha Gamer,20,2022-02-19T00:00:00Z,genshin impact|genshin impact mobile|genshin i...,382598,13241,0,1025,https://i.ytimg.com/vi/zk2rxLH80XA/default.jpg,False,False,Yae Miko surprised me in several ways and with...
2,yPYW5z7O1rw,Mo3 - For The Love (Official Visualizer),2022-02-14T20:00:29Z,UC_PQ7gGEKIwJv0YsnxH3dUw,HSM,10,2022-02-19T00:00:00Z,Mo3|Dallas|Texas|Rap Music|Badazz,373768,25484,0,1356,https://i.ytimg.com/vi/yPYW5z7O1rw/default.jpg,False,False,#Mo3 #ForTheLove #HSMOfficial Visualizer for ...
3,eEQMtIX61LA,"Bruno Mars, Anderson .Paak, Silk Sonic - Love'...",2022-02-14T20:00:09Z,UCoUM-UJ7rirJYP8CQ0EIaHA,Bruno Mars,10,2022-02-19T00:00:00Z,bruno mars|silk sonic|anderson .paak|anderson ...,1541464,85978,0,4419,https://i.ytimg.com/vi/eEQMtIX61LA/default.jpg,False,False,"Bruno Mars, Anderson .Paak, Silk Sonic - Love'..."
4,J6CbElkmmMA,GIANT ALEX Mod in Among Us...,2022-02-16T23:45:01Z,UCXHGoWpySS0smjanTUDa6Gw,GameToons Gaming,20,2022-02-19T00:00:00Z,among us|among us logic|among us adventures|ga...,989968,16910,0,516,https://i.ytimg.com/vi/J6CbElkmmMA/default.jpg,False,False,► SUBSCRIBE to GameToons Gaming! -https://www....
5,2bZYqFsU72Y,The First-Ever All-Electric Chevy Silverado – ...,2022-02-14T00:16:41Z,UCSVpCNZzOeMekuMiFze3fnQ,Chevrolet,2,2022-02-19T00:00:00Z,Chevrolet|Chevy|Silverado EV|Electric Silverad...,11553038,20700,0,3803,https://i.ytimg.com/vi/2bZYqFsU72Y/default.jpg,False,False,"With available Four-Wheel Steer, an available ..."
6,0z1dsxJcAPg,SURPRISING MY BOYFRIEND WITH HIS DREAM CAR !,2022-02-15T04:12:12Z,UCABbEMDC2EdsLuF0pkoV_cg,Jathaly,22,2022-02-19T00:00:00Z,new car|surprise|surprising my boyfriend|car g...,294989,17246,0,975,https://i.ytimg.com/vi/0z1dsxJcAPg/default.jpg,False,False,!!!!!! WELCOME TO JATHALY !!!!!Our solo channe...
7,wyWo_qkjlLk,How rich people vs really rich people act at t...,2022-02-16T16:31:41Z,UCJSICzUeXSxBvc0UAf2Up8g,Nicholas Crown,22,2022-02-19T00:00:00Z,[None],424249,50917,0,109,https://i.ytimg.com/vi/wyWo_qkjlLk/default.jpg,False,False,
8,xK5qVtczVmk,Toosii - Love Me Easy (Official Audio),2022-02-14T17:50:13Z,UCUtmzHuW43bFbUP-Xl0TTKA,Toosii 2x,10,2022-02-19T00:00:00Z,[None],458758,29118,0,1201,https://i.ytimg.com/vi/xK5qVtczVmk/default.jpg,False,False,Stream // Download Toosii’s “Pretty Girls Love...
9,VmI0YIqdjv8,I'M PREGNANT!,2022-02-14T14:56:53Z,UCF2oW5-MO8dB6ul9WH9xi0A,blndsundoll4mj,22,2022-02-19T00:00:00Z,[None],784231,39162,0,10912,https://i.ytimg.com/vi/VmI0YIqdjv8/default.jpg,False,False,


In [50]:
pd.read_sql(f"DROP TABLE IF EXISTS {db_name}.gb_csv", conn)


gb_csv_create = f"""
CREATE EXTERNAL TABLE IF NOT EXISTS {db_name}.gb_csv(
                video_id string,
                title string,
                publishedAt string,
                channelId string,
                channelTitle string,
                categoryId string,
                trending string,
                tags string,
                view_count string,
                likes string,
                dislikes string,
                comment_count string,
                thumbnail_link string,
                comments_disabled string,
                ratings_disabled string,
                description string)
            
                
                ROW FORMAT SERDE 'org.apache.hadoop.hive.serde2.OpenCSVSerde' 
                WITH SERDEPROPERTIES ("separatorChar" = ",") 
                LOCATION 's3://ads508youtubebucket/youtubeStatistics/csv/gb/'
                TBLPROPERTIES ("skip.header.line.count"="1");
"""

pd.read_sql(gb_csv_create, conn)

""


In [51]:
pd.read_sql(f"SELECT * FROM {db_name}.gb_csv LIMIT 10", conn)

,video_id,title,publishedat,channelid,channeltitle,categoryid,trending,tags,view_count,likes,dislikes,comment_count,thumbnail_link,comments_disabled,ratings_disabled,description
0,iLgCv4I6Bpk,Obsessed with Rebel Wilson hosting this year's...,2022-03-13T21:44:50Z,UCCj956IF62FbT7Gouszaj9w,BBC,24,2022-03-15T00:00:00Z,BBC iPlayer|United Kingdom|British TV|British ...,298981,4073,0,641,https://i.ytimg.com/vi/iLgCv4I6Bpk/default.jpg,False,False,Subscribe and 🔔 to the BBC 👉 https://bit.ly/BB...
1,lX0TPbCSAbM,"MORGENSHTERN - 12 (Official Video, 2022)",2022-03-13T22:00:31Z,UCWnqnojAgMdN0fQpr_xByJw,MORGENSHTERN,10,2022-03-15T00:00:00Z,12|morgenshtern|моргенштерн|клип|премьера|новы...,5191454,571067,0,40438,https://i.ytimg.com/vi/lX0TPbCSAbM/default.jpg,False,False,https://t.me/alisherhateu
2,Dx7zoJR17HE,"EATING 100,000 CALORIES WITH A SUMO WRESTLER!",2022-03-14T04:31:22Z,UC-O9o5coq8iFSVthivq__tw,Lexi Hensler,1,2022-03-15T00:00:00Z,Brent rivera|Lexi Rivera|ben azelart|Pierson w...,615766,31876,0,1234,https://i.ytimg.com/vi/Dx7zoJR17HE/default.jpg,False,False,"a bunch of friends and I attempted to eat 100,..."
3,zTcPgggZpfk,Rebel Wilson's Opening Monologue | EE BAFTA Fi...,2022-03-13T20:59:40Z,UCtggghgcffr7-OSjiRXRM3Q,BAFTA,24,2022-03-15T00:00:00Z,baftas|bafta film awards|bafta film awards 202...,143406,1166,0,201,https://i.ytimg.com/vi/zTcPgggZpfk/default.jpg,False,False,Rebel Wilson opens up proceedings at this year...
4,tR-E9XzwfNg,#POV: You can see how many days you have left ...,2022-03-13T12:09:35Z,UCt2efhikMJ_91ZYub1H5EcQ,Amara Chehade,1,2022-03-15T00:00:00Z,[None],1686074,210231,0,805,https://i.ytimg.com/vi/tR-E9XzwfNg/default.jpg,False,False,
5,zf7GbseHaFw,SHARING BIG NEWS ... Get Ready With Me!,2022-03-14T16:00:29Z,UC4qk9TtGhBKCkoWz5qGJcGg,Tati,26,2022-03-15T00:00:00Z,YouTube|Beauty|Makeup|Tutorial|Review|blogger|...,298205,21169,0,2203,https://i.ytimg.com/vi/zf7GbseHaFw/default.jpg,False,False,Big News / Life update today!xo's Tati😇 BEST ...
6,HmJRkaKnsKE,Making A Number 1 Song In 24 Hours,2022-03-13T19:13:49Z,UCKI_nC8ZYng9JmLcRbu1V_w,Stephen Tries Less,23,2022-03-15T00:00:00Z,stephen tries|stephen tries less|steven tries|...,99196,8710,0,294,https://i.ytimg.com/vi/HmJRkaKnsKE/default.jpg,False,False,This week we have less than one day to create ...
7,Yib8WJbcNoE,Brisket jerky,2022-03-12T23:50:43Z,UCyP9TINtZzvTJfN69GskqLQ,Dan-O’s Seasoning,26,2022-03-15T00:00:00Z,[None],546811,32602,0,157,https://i.ytimg.com/vi/Yib8WJbcNoE/default.jpg,False,False,
8,ep6gSdzQjyE,I Beat The Broad Jump World Record,2022-03-13T00:12:50Z,UC8cRojZvNtms970qonot1-w,Dexton Shorts,22,2022-03-15T00:00:00Z,I Beat The Broad Jump World Record|#Shorts,680176,47870,0,642,https://i.ytimg.com/vi/ep6gSdzQjyE/default.jpg,False,False,I Beat The Broad Jump World Record#Shorts
9,Pp7KqNkPqpA,Lacazette ends drought with pin-point penalty ...,2022-03-13T18:41:34Z,UCNAf1k0yIjyGu3k9BwAg3lg,Sky Sports Football,17,2022-03-15T00:00:00Z,sky sports|premier league|Football League|foot...,751371,11252,0,933,https://i.ytimg.com/vi/Pp7KqNkPqpA/default.jpg,False,False,SUBSCRIBE ► http://bit.ly/SSFootballSubPREMIER...


#### JSON Tables

### Data could also be accessed like this

In [6]:
bucket = "ads508youtubebucket"
data_key = "CA_youtube_trending_data.csv"
data_location_ca = "s3://{}/youtubeStatistics/csv/can/{}".format(bucket, data_key)

In [7]:
ca_df = pd.read_csv(data_location_ca)

As shown below, ingesting the data by directly accessing the bucket through Pandas creates a very clean dataset to work with.

In [8]:
ca_df.head()

,video_id,title,publishedAt,channelId,channelTitle,categoryId,trending_date,tags,view_count,likes,dislikes,comment_count,thumbnail_link,comments_disabled,ratings_disabled,description
0,KX06ksuS6Xo,Diljit Dosanjh: CLASH (Official) Music Video |...,2020-08-11T07:30:02Z,UCZRdNleCgW-BGUJf-bbjzQg,Diljit Dosanjh,10,2020-08-12T00:00:00Z,clash diljit dosanjh|diljit dosanjh|diljit dos...,9140911,296541,6180,30059,https://i.ytimg.com/vi/KX06ksuS6Xo/default.jpg,False,False,CLASH official music video performed by DILJIT...
1,J78aPJ3VyNs,I left youtube for a month and THIS is what ha...,2020-08-11T16:34:06Z,UCYzPXprvl5Y-Sf0g4vX-m6g,jacksepticeye,24,2020-08-12T00:00:00Z,jacksepticeye|funny|funny meme|memes|jacksepti...,2038853,353797,2628,40222,https://i.ytimg.com/vi/J78aPJ3VyNs/default.jpg,False,False,I left youtube for a month and this is what ha...
2,M9Pmf9AB4Mo,Apex Legends | Stories from the Outlands – “Th...,2020-08-11T17:00:10Z,UC0ZV6M2THA81QT9hrVWJG3A,Apex Legends,20,2020-08-12T00:00:00Z,Apex Legends|Apex Legends characters|new Apex ...,2381688,146740,2794,16549,https://i.ytimg.com/vi/M9Pmf9AB4Mo/default.jpg,False,False,"While running her own modding shop, Ramya Pare..."
3,3C66w5Z0ixs,I ASKED HER TO BE MY GIRLFRIEND...,2020-08-11T19:20:14Z,UCvtRTOMP2TqYqu51xNrqAzg,Brawadis,22,2020-08-12T00:00:00Z,brawadis|prank|basketball|skits|ghost|funny vi...,1514614,156914,5857,35331,https://i.ytimg.com/vi/3C66w5Z0ixs/default.jpg,False,False,SUBSCRIBE to BRAWADIS ▶ http://bit.ly/Subscrib...
4,VIUo6yapDbc,Ultimate DIY Home Movie Theater for The LaBran...,2020-08-11T15:10:05Z,UCDVPcEbVLQgLZX0Rt6jo34A,Mr. Kate,26,2020-08-12T00:00:00Z,The LaBrant Family|DIY|Interior Design|Makeove...,1123889,45803,964,2198,https://i.ytimg.com/vi/VIUo6yapDbc/default.jpg,False,False,Transforming The LaBrant Family's empty white ...


### NLP

The current dataframe is too large to implement NLP using Sklearn. The dataframes could be made smaller during ingestion phase by only using certain categories that our 'creators' exist in. 

In [ ]:

# stopwords from nltk
nltk.download()

In [3]:
stopwords = nltk.corpus.stopwords.words('english')
ps = nltk.PorterStemmer()

# 3 words to have more context
vec = CountVectorizer(min_df=1, analyzer='word', ngram_range=(3, 3))

We could then clean the text here and use Count Vectorizer from SkLearn to transform the long form text within the csvs to more manageable counts. This will make our data wide. Doing so will allow for transformation to Athena easier than transforming with strings. 

In [4]:
# Clean words from punctuation
def cleaning_text(text):
    text = "".join([word.lower() for word in text if word not in string.punctuation])
    tokens = re.split('\W+', text)
    text = [ps.stem(word) for word in tokens if word not in stopwords]
    return text

In [9]:
tag = ca_df['tags']

cleaning_text(tag)


['clash',
 'diljit',
 'dosanjh',
 'diljit',
 'dosanjh',
 'diljit',
 'dosanjh',
 'goat',
 'album',
 'diljit',
 'dosanjh',
 'new',
 'album',
 'punjabi',
 'song',
 '2020',
 'punjabi',
 'new',
 'song',
 'new',
 'song',
 '2020',
 'goat',
 'diljit',
 'dosanjh',
 'kidd',
 'punjabi',
 'music',
 'kidd',
 'music',
 'raj',
 'ranjodh',
 'song',
 'goat',
 'diljit',
 'dosanjh',
 'full',
 'album',
 'diljit',
 'dosanjh',
 'karan',
 'aujla',
 'song',
 'diljit',
 'dosanjh',
 'new',
 'song',
 'diljit',
 'dosanjh',
 'song',
 'goat',
 'diljit',
 'dosanjh',
 '2020',
 'goat',
 '2020',
 'latest',
 'punjabi',
 'song',
 '2020',
 'punajbi',
 '2020',
 'latest',
 'song',
 'punjabi',
 'song',
 'punjabi',
 'new',
 'song',
 'punjabi',
 'clashjacksepticey',
 'funni',
 'funni',
 'meme',
 'meme',
 'jacksepticey',
 'meme',
 'reddit',
 'subreddit',
 'commun',
 'commun',
 'meme',
 'commun',
 'subreddit',
 'jacksepticey',
 'subreddit',
 'reddit',
 'meme',
 'fan',
 'submit',
 'spici',
 'meme',
 'funni',
 'pic',
 'reaction',


In [10]:
tags = vec.fit_transform(tag)

In [11]:
print(tags.shape)

(189143, 608970)


In [12]:
vec.get_feature_names()

['00 00z sp',
 '00 2k21 demo',
 '00 agent 00',
 '00 agent00 fanum',
 '00 amp agent',
 '00 amp duke',
 '00 brucedropemoff agent',
 '00 chrisnxtdoor imdavisss',
 '00 duke dennis',
 '00 imdavisss chrisnxtdoor',
 '00 imdavisss fanum',
 '00 kai cenat',
 '00 loona midnight',
 '00 midnight 이달의',
 '00 myteam mynba',
 '00 nba 2k22',
 '00 the prince',
 '00 이달의 소녀',
 '000 000 le',
 '000 000 villagers',
 '000 challenge last',
 '000 date dating',
 '000 euros inox',
 '000 le dernier',
 '000 misterbeast mrbeast',
 '000 talent fortnite',
 '000 valouzz lebouseuh',
 '000 villagers worship',
 '00001 odds challenge',
 '00001 trick shots',
 '001 001 prototype',
 '001 action adventure',
 '001 and 456',
 '001 containment containment',
 '001 fiel jhayco',
 '001 gate guardian',
 '001 origin story',
 '001 prototype prototype',
 '001 scp 001',
 '001 scp001 scp',
 '0012000003 pass assist',
 '0012000004 clippers la',
 '0012000006 pass assist',
 '0012000014 pass assist',
 '0012000020 toronto kyle',
 '0012000021 new